In [1]:
from openai import OpenAI
import os
import json
# from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"] })

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [3]:
os.getenv("RUNPOD_TOKEN")

'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'

In [4]:
client = OpenAI(
            api_key=os.getenv("RUNPOD_TOKEN"),
            base_url=os.getenv("RUNPOD_CHATBOT_URL"),
        )
model_name = os.getenv("MODEL_NAME")

In [5]:
model_name

'llama-3.3-70b-specdec'

# Get LLM response

In [6]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome.


In [7]:
# messages = [{'role':'user','content':"write a snnake game"}]
# response = get_chatbot_response(client,model_name,messages)
# print(response)

# Prompt engineering

## Structred output

In [40]:
# student_raw_text ="""The OSI model in networking is a conceptual framework used to understand and standardize the functions of a communication system. It consists of seven layers: Physical, Data Link, Network, Transport, Session, Presentation, and Application.
# A router is a device that forwards data packets between computer networks, while a switch operates at a local level to connect devices within a single network.
# Firewalls are crucial for network security because they monitor and control incoming and outgoing network traffic based on security rules, helping prevent unauthorized access and attacks."""


# teacher_questions = """1. What is the OSI model in networking?
# 2. What are the differences between a router and a switch?
# 3. Explain the importance of firewalls in network security."""

# teacher_raw_text = """
# 1. The OSI (Open Systems Interconnection) model is a conceptual framework used to standardize network communication. It consists of seven layers: 
#    - **Physical Layer**: Handles the physical connection between devices.  
#    - **Data Link Layer**: Manages node-to-node communication and error detection.  
#    - **Network Layer**: Handles routing and forwarding of data packets.  
#    - **Transport Layer**: Ensures reliable transmission of data between systems.  
#    - **Session Layer**: Manages sessions and connections between applications.  
#    - **Presentation Layer**: Translates data formats and encryption.  
#    - **Application Layer**: Provides network services to end-users.

# 2. A **router** and a **switch** serve different functions in networking:
#    - A **router** operates at the **network layer (Layer 3)** and is responsible for forwarding data between different networks using IP addresses.  
#    - A **switch** operates at the **data link layer (Layer 2)** and is used within a local network (LAN) to connect devices efficiently using MAC addresses.  
#    - Routers enable communication between networks (e.g., connecting to the internet), while switches improve network efficiency within a single network.

# 3. Firewalls play a critical role in **network security** by:
#    - Monitoring and controlling **incoming and outgoing traffic** based on predefined security rules.  
#    - Preventing **unauthorized access** by blocking malicious traffic.  
#    - Acting as a barrier between a **trusted internal network** and **untrusted external sources**, reducing the risk of cyber threats.  
#    - Implementing security policies such as **packet filtering, intrusion detection, and deep packet inspection** to safeguard sensitive data.
# """



student_raw_text = """Alice has 12 apples, and she gives 4 apples to her friend Bob. Then, she buys 6 more apples from a store. After that, she equally divides all her apples among 3 of her friends.

A train travels at a speed of 60 km/h. If it travels for 3 hours without stopping, how far does it go?"""

teacher_questions = """1. How many apples does Alice have after giving some to Bob and buying more?
2. How many apples does each friend get after Alice divides them equally?
3. How far does the train travel in 3 hours if it moves at 60 km/h?
4. How many oranges does Charlie have left after eating some and receiving more from a friend?
"""
teacher_raw_text = """ 
1. Alice starts with 12 apples. She gives 4 to Bob, leaving her with:
   12 - 4 = 8 apples.
   Then, she buys 6 more apples:
   8 + 6 = 14 apples.
   So, Alice has **14 apples** after the transactions.

2. Alice equally divides the 14 apples among 3 friends:
   14 ÷ 3 ≈ 4.67 apples per friend.
   Since apples are whole objects, she can distribute **4 apples per friend** and have **2 apples left**.

3. The train travels at a speed of **60 km/h** for **3 hours**. The total distance covered is:
   Distance = Speed × Time
   Distance = 60 km/h × 3 h = **180 km**.
   So, the train travels **180 km** in 3 hours.

4. Charlie starts with **10 oranges**. He eats **3 oranges**, leaving him with:  
   10 - 3 = **7 oranges**.  
   Then, his friend gives him **5 more oranges**:  
   7 + 5 = **12 oranges**.  
   So, Charlie has **12 oranges** after the transactions.
"""


## fomrating 

In [38]:
system_prompt = """
You are an AI assistant tasked with formatting student assignments based on teacher-provided questions and responses, along with the student's raw text. Your goal is to structure the student’s answers correctly in relation to the teacher's response.

### Instructions:
1. Given a set of questions, a raw student response, and a teacher's response, identify the answers in the student’s text.
2. For each question:
   - Locate the corresponding answer in the student's response.
   - Compare it with the teacher's response to ensure completeness.
3. Return a structured list of dictionaries. Each dictionary must contain:
   - `"Question"`: The exact question provided by the teacher.
   - `"Student Answer"`: The student's response to the question.
   - `"Teacher Answer"`: The teacher's correct response for reference.
4. If the student has not provided an answer, set the `"Student Answer"` field to an empty string or indicate "Unanswered."
5. Ensure the output is clean and structured, with each question-answer pair correctly aligned.

### Format Example:
**Teacher’s Questions:**
1. What is the capital of France?  
2. Who is the president of the USA?  

**Student's Raw Text:**  
"The capital of France is Paris. The USA has a leader named Biden."  

**Teachers's Raw Text:**  
"The capital of France is Paris. The USA has a leader named Biden."  

**Teacher's Response:**  
"The capital of France is **Paris**. The president of the USA is **Joe Biden**."  

**Output:**
The system will return the following JSON object, you are not alllowed to return anyhitng else other than the JSON object. and don't include this as well ```json```:
```json
{
    "teacher_formated_response":[
      {
        "question": "What is the capital of France?",
        "answer": "The capital of France is Paris."
      },
      {
        "question": "Who is the president of the USA?",
        "answer": "Joe Biden is the president of the USA."
      }
    ],

    "student_formated_response": [
      {
        "question": "What is the capital of France?",
        "answer": "The capital of France is Paris."
      },
      {
        "question": "Who is the president of the USA?",
        "answer": "Joe Biden is the president of the USA."
      }
    ]
}
```
"""




user_prompt = f"""
Teacher’s Questions:
{teacher_questions}

Student Raw Text:
{student_raw_text}

Teacher Raw Text:
{teacher_raw_text}
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
student_response = get_chatbot_response(client,model_name,messages)
print(student_response)

{
    "teacher_formated_response":[
      {
        "question": "How many apples does Alice have after giving some to Bob and buying more?",
        "answer": "Alice has 14 apples after the transactions."
      },
      {
        "question": "How many apples does each friend get after Alice divides them equally?",
        "answer": "Each friend gets 4 apples."
      },
      {
        "question": "How far does the train travel in 3 hours if it moves at 60 km/h?",
        "answer": "The train travels 180 km in 3 hours."
      },
      {
        "question": "How many oranges does Charlie have left after eating some and receiving more from a friend?",
        "answer": "Charlie has 12 oranges after the transactions."
      }
    ],

    "student_formated_response": [
      {
        "question": "How many apples does Alice have after giving some to Bob and buying more?",
        "answer": "Alice has 12 apples, gives 4 to Bob, and buys 6 more, so she has 14 apples."
      },
      {
       

In [41]:
out = json.loads(student_response)
out["student_formated_response"]


[{'question': 'How many apples does Alice have after giving some to Bob and buying more?',
  'answer': 'Alice has 12 apples, gives 4 to Bob, and buys 6 more, so she has 14 apples.'},
 {'question': 'How many apples does each friend get after Alice divides them equally?',
  'answer': 'Alice divides all her apples among 3 of her friends, but the exact distribution is not specified.'},
 {'question': 'How far does the train travel in 3 hours if it moves at 60 km/h?',
  'answer': "The train travels at a speed of 60 km/h for 3 hours, but the exact distance is not calculated in the student's response."},
 {'question': 'How many oranges does Charlie have left after eating some and receiving more from a friend?',
  'answer': 'Unanswered'}]

In [22]:
system_prompt = """
You are an AI assistant tasked with formating the student assignments based on teacher-provided questions and the raw student text. For each question, you need to find the corresponding answer in the raw text and return it in a structured format.

Instructions:
1. Given a set of questions and a raw student text, identify the answers in the student's response.
2. For each question, locate the part of the raw text that corresponds to the answer.
3. Return a list of dictionariessystem. Each dictionary should contain:
   - "Question": The exact question provided by the teacher.
   - "Answer": The student's answer to that specific question from the raw text.
4. If a question is not answered by the student, leave the answer field empty or indicate that the question is unanswered.
5. Ensure the output is structured and clean, and each question-answer pair should be matched correctly.

Format Example:
Teacher’s Questions:
1. What is the capital of France?
2. Who is the president of the USA?

Student's Raw Text:
"The capital of France is Paris. Joe Biden is the president of the USA."

Output:
The system will return the answers in the following format:

[
  {
    "question": "What is the capital of France?",
    "answer": "The capital of France is Paris."
  },
  {
    "question": "Who is the president of the USA?",
    "answer": "Joe Biden is the president of the USA."
  }
]

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
"""

user_prompt = f"""student_raw_text
Teacher’s Questions:
{teacher_questions}

Raw Text:
{student_raw_text}
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
student_response = get_chatbot_response(client,model_name,messages)
print(student_response)

[
  {
    "question": "How many apples does Alice have after giving some to Bob and buying more?",
    "answer": "Alice has 12 apples, and she gives 4 apples to her friend Bob. Then, she buys 6 more apples from a store."
  },
  {
    "question": "How many apples does each friend get after Alice divides them equally?",
    "answer": "After that, she equally divides all her apples among 3 of her friends."
  },
  {
    "question": "How far does the train travel in 3 hours if it moves at 60 km/h?",
    "answer": "A train travels at a speed of 60 km/h. If it travels for 3 hours without stopping, how far does it go?"
  },
  {
    "question": "How many oranges does Charlie have left after eating some and receiving more from a friend?",
    "answer": ""
  }
]


In [23]:
user_prompt = f"""
Teacher’s Questions:
{teacher_questions}

Raw Text:
{teacher_raw_text}
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
teacher_response = get_chatbot_response(client,model_name,messages)
print(teacher_response)

[
  {
    "question": "How many apples does Alice have after giving some to Bob and buying more?",
    "answer": "So, Alice has **14 apples** after the transactions."
  },
  {
    "question": "How many apples does each friend get after Alice divides them equally?",
    "answer": "Since apples are whole objects, she can distribute **4 apples per friend** and have **2 apples left**."
  },
  {
    "question": "How far does the train travel in 3 hours if it moves at 60 km/h?",
    "answer": "So, the train travels **180 km** in 3 hours."
  },
  {
    "question": "How many oranges does Charlie have left after eating some and receiving more from a friend?",
    "answer": "So, Charlie has **12 oranges** after the transactions."
  }
]


In [24]:
system_prompt = """
You are an AI agent tasked with distributing the total marks across a set of questions based on their difficulty, length, and importance. The input consists of a structured response containing question-answer pairs.

Instructions:
1. The input will consist of a list of dictionaries, each containing a "question" and an "answer".
2. The total marks for the assignment will be provided by the teacher.
3. Distribute the total marks across the questions, ensuring that:
   - More difficult questions receive more marks.
   - Longer answers or more detailed answers are awarded more marks.
   - Ensure the marks are distributed in a way that reflects the importance and complexity of each question.
4. Return the output as a dictionary where:
   - The keys are the question numbers (e.g., "1", "2", "3", etc.).
   - The values are the assigned marks for each question.

Example Input:
[
  {"question": "What is the OSI model in networking?", "answer": "The OSI model in networking is a conceptual framework..."},
  {"question": "What are the differences between a router and a switch?", "answer": "A router forwards data packets..."},
  {"question": "Explain the importance of firewalls in network security.", "answer": "Firewalls are crucial for network security because..."}
]

total_marks = 30

Example Output:
Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
  "Q1": 8,
  "Q2": 7,
  "Q3": 15
}


"""
total_marks = 13
user_prompt = f"""
Here is the structured response from the previous agent containing the teacher's questions and the student's answers:
{teacher_response}

and the total marks for the assignment is {total_marks}.
"""


messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
rubric_response = get_chatbot_response(client,model_name,messages)
print(rubric_response)

{
  "Q1": 3,
  "Q2": 2,
  "Q3": 4,
  "Q4": 4
}


In [28]:
system_prompt = """
You are an AI-based assignment evaluator designed to assess student responses by comparing them against teacher-provided answers and a grading rubric, rubric contain the totals marks for each question. Your task is to analyze structured inputs, assign marks to each question, and generate personalized feedback for the entire assignment.

Inputs:
1. Extracted Questions from Student File – A structured list of questions answered by the student.
2. Teacher’s Answers – The correct or expected responses in a structured format.
3. Rubric Details – A structured guideline on how marks should be distributed per question.

Evaluation Process:
- Compare each student’s response with the corresponding teacher’s answer.
- Use the rubric to fairly assign marks to each question.
- Ensure the grading considers relevance, correctness, and completeness.
- Generate a final response with question-wise marks and overall personalized feedback.


Guidelines:
- Keep the feedback constructive and encouraging.
- Ensure fair grading based on the rubric.
- Do not include total marks; the user will handle that separately.



Example Input:

Student's Extracted Questions and Answers (Structured Format):
[
  {"question": "What is the OSI modCharlie starts with 10 oranges. He eats 3 oranges, leaving him with:
10 - 3 = 7 oranges.
Then, his friend gives him 5 more oranges:
7 + 5 = 12 oranges.
So, Charlie has 12 oranges after the transactions.el in networking?", "answer": "The OSI model in networking is a conceptual framework..."},
  {"question": "What are the differences between a router and a switch?", "answer": "A router forwards data packets..."},
  {"question": "Explain the importance of firewalls in network security.", "answer": "Firewalls are crucial for network security because..."}
]

Teacher's Extracted Questions and Answers (Structured Format):

[
  {"question": "What is the OSI model in networking?", "answer": "The OSI model is a conceptual framework..."},
  {"question": "What are the differences between a router and a switch?", "answer": "A router and a switch serve different functions..."},
  {"question": "Explain the importance of firewalls in network security.", "answer": "Firewalls play a critical role in network security by..."}
]




Rubric Details (Structured Format):

{
  "Q1": 3,
  "Q2": 3,
  "Q3": 4
  ...
}


Example Output:

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object (start and end with curlly braces):

Output Format (JSON):
{   
    "chain_of_thought": "<here you will think during evaluation process, consider each answers key points>",
    "marks": {
        "Q1": <marks_awarded>,
        "Q2": <marks_awarded>,
        "Q3": <marks_awarded>,
        ...
    },
    "personalized_feedback": "<feedback for student based on his assignment, use 2nd person approach>"
}
"""

user_prompt = """
Student's Extracted Questions and Answers (Structured Format):
{student_response}

Teacher's Extracted Questions and Answers (Structured Format):
{teacher_response}

Rubric Details (Structured Format):
{rubric_response}
"""


messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
eval_response = get_chatbot_response(client,model_name,messages)
print(eval_response)

```json
{
    "chain_of_thought": "To evaluate the student's assignment, I will compare each response with the corresponding teacher's answer and use the rubric to assign marks. I will consider relevance, correctness, and completeness for each question. For Q1, the student's answer is 'The OSI model in networking is a conceptual framework...' which is similar to the teacher's answer 'The OSI model is a conceptual framework...'. For Q2, the student's answer is 'A router forwards data packets...' which partially matches the teacher's answer 'A router and a switch serve different functions...'. For Q3, the student's answer is 'Firewalls are crucial for network security because...' which is close to the teacher's answer 'Firewalls play a critical role in network security by...'. Based on the rubric, I will assign marks accordingly.",
    "marks": {
        "Q1": 2.5,
        "Q2": 2,
        "Q3": 3
    },
    "personalized_feedback": "You have demonstrated a good understanding of the OSI 

In [19]:
json_response = json.loads(eval_response)
json_response

{'chain_of_thought': "To evaluate the student's assignment, I will compare each response with the corresponding teacher's answer and use the rubric to assign marks. For Q1, the student's answer 'The OSI model in networking is a conceptual framework...' is similar to the teacher's answer 'The OSI model is a conceptual framework...'. The key points in the student's answer are the definition of the OSI model. For Q2, the student's answer 'A router forwards data packets...' is partially correct, but it does not fully address the differences between a router and a switch. The key points in the student's answer are the function of a router. For Q3, the student's answer 'Firewalls are crucial for network security because...' is similar to the teacher's answer 'Firewalls play a critical role in network security by...'. The key points in the student's answer are the importance of firewalls in network security.",
 'marks': {'Q1': 2.5, 'Q2': 1.5, 'Q3': 3.5},
 'personalized_feedback': 'You have de

## input structuring

[
{
    "country": "Italy",
    "capital": "Rome"
},
{
    "country": "France",
    "capital": "Paris"
},
{
    "country": "Germany",
    "capital": "Berlin"
}
]


In [10]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Give the model time to think (Chain of thought)

> https://arxiv.org/pdf/2205.11916

In [12]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [13]:
259/2*8654+91072*33-12971

4113098.0

In [19]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1434921119
}


In [20]:
4113098.0 - 1434921119.0

-1430808021.0

In [21]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [22]:
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121111
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121111 and 3005016: 1121111 + 3005016 = 4126137
    5. Subtract 12971 from 4126137: 4126137 - 12971 = 4114026
    ",
    "result": 4114026
}


In [23]:
4113098.0 - 4114026

-928.0

# RAG - Retrieval Augmented Generation

#### Asking about a subject that the LLM does not know anything about

In [11]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

Since I'm not aware of any official information about the iPhone 16, I'll provide you with some general information about the latest iPhone models and some rumored features that might be included in future iPhone models.

However, I can tell you about the latest iPhone models, such as the iPhone 14 series, which includes:

1. iPhone 14
2. iPhone 14 Plus
3. iPhone 14 Pro
4. iPhone 14 Pro Max

Some of the notable features of the iPhone 14 series include:

1. Improved cameras with a new 48MP main camera on the Pro models
2. Faster A16 Bionic chip
3. Longer battery life
4. New colors and designs
5. Enhanced display with a higher refresh rate

As for the iPhone 16, there's no official information available yet. However, based on rumors and leaks, here are some potential features that might be included:

1. Improved cameras with a new periscope lens or a higher-resolution main camera
2. Faster A17 Bionic chip or a new chip design
3. Enhanced display with a higher refresh rate or a new OLED p

#### Giving Context to the unknown subject

In [12]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [13]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch
   - iPhone 16 Plus: 6.7-inch
   - iPhone 16 Pro: 6.3-inch
   - iPhone 16 Pro Max: 6.9-inch

2. Thinner bezels and a more durable Ceramic Shield.

3. New A18 chip (A18 Pro for Pro models) for improved performance, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks.

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor.
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.

5. Advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording.

6. Extended battery life, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone.

7. Switch to USB-C for

#### Automatically extract context data from database

In [27]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [28]:
data = [iphone_16,samsung_s23]

In [29]:
user_prompt = """What's new in iphone 16?"""

In [30]:
embedding_client = OpenAI(
        api_key=os.getenv("RUNPOD_TOKEN"), 
        base_url=os.getenv("RUNPOD_EMBEDDING_URL")
    )

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [12]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [13]:
user_prompt_embeddings = get_embedding(embedding_client,model_name,user_prompt)

In [14]:
user_prompt_embeddings

[[-0.05110831558704376,
  -0.03489653766155243,
  0.06350376456975937,
  -0.008884420618414879,
  -0.03724740073084831,
  -0.042040713131427765,
  0.015921736136078835,
  0.031126001849770546,
  -0.01283814013004303,
  0.04246814176440239,
  0.05556579306721687,
  0.01331899780780077,
  -0.07211340963840485,
  0.0014378030318766832,
  0.09635474532842636,
  0.028714081272482872,
  0.11510057002305984,
  -0.14752411842346191,
  -0.07907439768314362,
  0.016150716692209244,
  -0.0367589071393013,
  -0.014563122764229774,
  -0.03975091129541397,
  -0.030057430267333984,
  0.030118491500616074,
  0.06139714643359184,
  0.0010905168019235134,
  -0.012731282971799374,
  -0.019188515841960907,
  -0.1384870558977127,
  -0.00042075058445334435,
  -0.006739642005413771,
  0.0166850034147501,
  0.026943303644657135,
  0.000493737927172333,
  -0.03840756416320801,
  -0.0031103105284273624,
  0.03489653766155243,
  -0.009601891040802002,
  0.06374800950288773,
  0.029599469155073166,
  0.0912255942

In [15]:
user_prompt_embeddings = user_prompt_embeddings[0]

In [16]:
data_embeddings = [get_embedding(embedding_client,model_name,x)[0] for x in data]

In [17]:
data_similaraty_scores = cosine_similarity([user_prompt_embeddings], data_embeddings)

In [20]:
data_similaraty_scores

array([[0.73779758, 0.52622817]])

In [22]:
closest_entry_index=data_similaraty_scores.argmax()
closest_entry_index

0

In [23]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [24]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [25]:
user_prompt_with_data

'\n\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine

In [26]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch
   - iPhone 16 Plus: 6.7-inch
   - iPhone 16 Pro: 6.3-inch
   - iPhone 16 Pro Max: 6.9-inch

2. Thinner bezels and a more durable Ceramic Shield.

3. New A18 chip (A18 Pro for Pro models) for improved performance, with:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max.

7. Switch to USB-C for faster charging and data transfer.

8. Support for up to 2x faster video encoding in Pro models.

9. Starting prices remain consistent with previous generations:
   - iPhone 16: $799
 